In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter



In [2]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./half_month_collections.pkl")

In [3]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [20]:
def get_compatibility_dict(L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket]):
    """ function to get a dict of compatible markets for each asset

    Args:
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe

    Returns:
        dict: list of compatible loads for each market
    """
    compatible_dict = {}
    for market in M:
        #print(type(market))
        asset_list = []
        for asset in L:
            if asset.response_time <= market.response_time and market.area == asset.area:
                if market.direction == "up":
                    if asset.direction != "down":
                        asset_list.append(asset)
                elif market.direction == "down":
                    if asset.direction != "up":
                        asset_list.append(asset)
                else:
                        asset_list.append(asset)
            
        compatible_dict[market] = asset_list
    return compatible_dict


In [21]:
compatible_dict = get_compatibility_dict(L = L ,M = M)

In [22]:
M[36].name

'RK_up_NO5'

In [23]:
len(compatible_dict[M[26]])

889

In [24]:
def get_possible_dates(date : pd.Timestamp):
    """ Function to get the possible dates for placing a bid given the current date

    Args:
        date (pd.Timestamp): the current date

    Returns:
        (pd.date_range, str): the possible dates for placing a bid and for which market
    """
    if date.hour == 17: # FCR D-2
        return (pd.date_range(date + timedelta(days=1) + timedelta(hours=7), date + timedelta(days = 2) + timedelta(hours = 6), freq='H', tz = "Europe/Oslo"), "D_2")
    elif date.hour == 7: # aFRR
        return (pd.date_range(date + timedelta(hours = 17), date + timedelta(days = 1) + timedelta(hours = 16), freq='H', tz = "Europe/Oslo"), "aFRR")
    elif date.hour == 18: # FCR D-1
        return (pd.date_range(date + timedelta(hours=6), date + timedelta(days = 1) + timedelta(hours = 5), freq='H', tz = "Europe/Oslo"), "D_1")
    else:
        return ([], "No bids")

In [25]:
def get_expected_prices_for_hours(possible_hours : pd.Timestamp, area : str, direction : str):
    """ Function to find the expected price for a given hour, area and direction. This function should be updated to be smarter in the future.

    Args:
        hour (pd.Timestamp): hour for which the expected price is calculated
        area (str): area for which the expected price is calculated
        direction (str): direction for which the expected price is calculated

    Returns:
        float: expected price for the given hour, area and direction
    """
    markets_to_check = [market for market in markets_name_dict.values() if market.area == area and market.direction == direction]
    return np.array([np.mean([market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in markets_to_check]) for hour in possible_hours])
    

In [60]:
def check_constraints(possible_assets : [[new_meters.PowerMeter]], possible_hours : [pd.Timestamp], possible_volumes : np.array, expected_prices : np.array, market : final_markets.ReserveMarket):
            
    max_vol = [market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for hour in possible_hours] # set of volumes for markets
    constrained_vols = [vol if vol >= market.min_volume and vol < max_vol[index] else 0 for index, vol in enumerate(possible_volumes)] #works
    # get the prices for the given market within the given hours
    possible_prices = [market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for hour in possible_hours] # set of prices for markets
    # Calculate the possible revenues
    """print(f"possible_prices in function : {possible_prices}")
    print(f"expected_prices in function : {expected_prices}")"""
    bids_to_be_made = np.where(possible_prices >= expected_prices, possible_prices, 0)
    """print(f"bids_to_be_made in function : {bids_to_be_made}")
    print(f"possible_volumes in function : {constrained_vols}")"""
    possible_revenues = constrained_vols * bids_to_be_made
    
    #print(f"possible_revenues in function : {possible_revenues}")
    possible_assets = [assets if possible_revenues[index] > 0 else [] for index, assets in enumerate(possible_assets)]
    #print(f"possible_assets in function : {possible_assets}")
    return possible_revenues, possible_assets

In [61]:
def place_bids(possible_hours : [pd.Timestamp], available_assets : dict, market : final_markets.ReserveMarket):
    """ Function to place bids for a given market and set of hours. The bids are placed for every hour in the set of hours. The bids are placed for the assets that are not already bid in to other markets in the given hours. The bids are placed for the assets that are compatible with the given market.

    Args:
        possible_hours (pd.Timestamp]): The time stamps for which the bids are placed
        available_assets (dict): Dictionary with the available assets for each hour
        market (final_markets.ReserveMarket): the market to be bid in to

    Returns:
        tuple (np.array, np.array): The possible revenue of the placed bids for each hour and the assets which are bid for each hour
    """
    expected_prices = get_expected_prices_for_hours(possible_hours, market.area, market.direction)
    hourly_assets = [available_assets[hour] for hour in possible_hours]
    possible_assets = [[asset for asset in hourly_assets[h] if asset in compatible_dict[market]] for h in range(len(possible_hours))]

    #print(f"expected_pries: {expected_prices}")
    """print(f"hourly_assets: {hourly_assets}")
    print(f"possible_assets: {len(possible_assets)}")"""
    if market.direction == "up":
        # check the compatibility for the assets
        possible_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        #print(f"possible_volumes: {possible_volumes}")
        possible_revenues, possible_assets = check_constraints(possible_assets, possible_hours,possible_volumes,  expected_prices, market)
    elif market.direction == "down":
        possible_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        #print(f"possible_volumes: {possible_volumes}")

        possible_revenues, possible_assets = check_constraints(possible_assets, possible_hours, possible_volumes, expected_prices, market)
    else:        
        possible_up_volumes = [sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        possible_down_volumes = [sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in possible_assets[h]]) for h, hour in enumerate(possible_hours)]
        #print(f"possible_up_volumes: {possible_up_volumes}")
        #print(f"possible_down_volumes: {possible_down_volumes}")
        # possible volumes should be != 0 if both up and down volume is higher than min_volume. If both vols are higher than min_volume, then the actuale volume should be decided by the dominant direction
        indices = [H.get_loc(ts) for ts in possible_hours]
        directions = np.array(dominant_directions[indices[0]:indices[-1]+1])
        actual_volumes = np.where(directions == "up", possible_up_volumes, possible_down_volumes)
        #print(f"actual_volumes: {actual_volumes}")
        # Find the hours where both up and down volume is higher than min_volume
        possible_volumes = [actual_volumes[h] if possible_up_volumes[h] >= market.min_volume and possible_down_volumes[h] >= market.min_volume else 0 for h in range(len(possible_hours))]
        #possible_volumes = np.where((possible_up_volumes >= min_vols and possible_down_volumes >= min_vols), actual_volumes, 0) 
        #print(f"possible_volumes: {possible_volumes}")
        possible_revenues, possible_assets = check_constraints(possible_assets, possible_hours, possible_volumes, expected_prices, market)
        """print(f"possible_revenues: {possible_revenues}")
        print(f"possible_assets: {possible_assets}")"""
    return (possible_revenues, possible_assets)


In [62]:
(possible_hours, market_name) = get_possible_dates(pd.Timestamp(2023, 6, 14, 7))

test = place_bids(possible_hours = possible_hours, available_assets = {hour: L.copy() for hour in possible_hours}, market = markets_name_dict["aFRR down_NO1"])

In [63]:
def get_possible_bids( H : [pd.Timestamp], L : [new_meters.PowerMeter], M : [final_markets.ReserveMarket]):
    """ Function to find the possible bids for the given input hours, assets and markets. where the possible bids are not constrained to be for every one of the 24 hours.

    Args:
        H (DateTimeIndex): list of the possible hours where bids can be placed
        L ([new_meters.PowerMeter]): list of all assets
        M ([final_markets.ReserveMarket]): list of all markets

    Returns:
        dict: dictionary with the possible bids for each market where the keys are tuples of the market name and the date and the values are tuples of the possible revenues and the assets that can bid in the market
    """
    bid_timeframe = H.append(pd.date_range(H[-1] + timedelta(hours=1), periods=48, freq='H', tz="Europe/Oslo"))
    available_assets = {hour: L.copy() for hour in bid_timeframe}
    bids = {}
    revenue = 0

    for hour in H:
        (possible_hours, market_name) = get_possible_dates(hour)
        if len(possible_hours) != 24:
            #print(f"No bids for {hour}")
            #print(f"possible_hours: {len(possible_hours)}")
            continue

        possible_markets = [m for m in M if market_name in m.name]
        for market in possible_markets:
            (possible_revenues, possible_assets) = place_bids(market=market, available_assets=available_assets, possible_hours=possible_hours)
            revenue += np.sum(possible_revenues)

            for h, bid_hour in enumerate(possible_hours):
                # Select assets with positive revenue for bidding
                assets_to_bid = [asset for asset in possible_assets[h] if possible_revenues[h] > 0]
                # Flatten the list if it's a list of lists
                #assets_to_bid = [item for sublist in assets_to_bid for item in sublist]
                # Store the bid information
                bids[(market.name, bid_hour)] = [asset.meter_id for asset in assets_to_bid]
                # Update available assets
                available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in assets_to_bid]

    return bids, revenue, available_assets
            

In [64]:
bids, revenue, av_assets  = get_possible_bids(H[:24], L, M)

In [65]:
revenue

3708.9500603600013

In [66]:
markets= [market[0] for market in bids.keys()]
hours = [market[1] for market in bids.keys()]



In [70]:
def feasibility_check(bids, L):
    """ Function to check if the bids are feasible. The bids are feasible if there are no duplicates in the bids for each hour."""
    markets= set([market[0] for market in bids.keys()])
    #print(f"Markets: {markets}")
    hours = set([market[1] for market in bids.keys()])
    #print(f"Hours: {hours}")
    keys = list(bids.keys())
    #print(f"Keys: {keys}")
    hourly_assets = {}
    asset_meter_id_dict = {asset.meter_id: asset for asset in L}
    for hour in hours:
       # print(f"Hour: {hour}")
        hourly_as_list = []
        for market_name in markets:
            #print(f"Market: {market}")
            if (market_name, hour) in keys:
                hourly_as_list.append(bids[(market_name, hour)])
                assets = bids[(market_name, hour)]
                market = markets_name_dict[market_name]
                assets = [asset_meter_id_dict[asset] for asset in assets]
                if len(assets) > 0:
                    assert len(assets) == len(set(assets)), f"Duplicate assets in hour {hour}"
                    if market.direction == "up":
                        assert all([asset.direction != "down" for asset in assets]), f"Down assets in up market {market.name} in hour {hour}"
                        total_flex_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] for asset in assets])
                    elif market.direction == "down":
                        assert all([asset.direction != "up" for asset in assets]), f"Up assets in down market in hour {hour}"
                        total_flex_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] for asset in assets])
                    else:
                        if dominant_directions[H.get_loc(hour)] == "up":
                            #assert all([asset.direction != "down" for asset in assets]), f"Down assets in up market in hour {hour}"
                            total_flex_volume = sum([asset.up_flex_volume["value"].loc[asset.up_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "down" else 0 for asset in assets])
                        elif dominant_directions[H.get_loc(hour)] == "down":
                            #assert all([asset.direction != "up" for asset in assets]), f"Up assets in down market in hour {hour}"
                            total_flex_volume = sum([asset.down_flex_volume["value"].loc[asset.down_flex_volume["Time(Local)"] == hour].values[0] if asset.direction != "up" else 0 for asset in assets])
                    assert total_flex_volume >= market.min_volume, f"Total flex volume {total_flex_volume} is less than the minimum volume {market.min_volume} in hour {hour} for market {market.name}"
                    total_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
                    assert total_flex_volume <= total_max_volume, f"Total flex volume {total_flex_volume} is more than the maximum volume {total_max_volume} in hour {hour}"
                    assert all([asset.response_time <= market.response_time for asset in assets]), f"Asset with response time higher than the market {market.name} response time in hour {hour}"
                            
        # Add the flattened list to the dictionary
        hourly_assets[hour] = [item for sublist in hourly_as_list for item in sublist]
    
    #print(hourly_assets)
    for hour in hours:
        if hourly_assets[hour] == []:
            continue
        else:
            """print(f"Hour : {hour}")
            print(f"type of hour object : {type(hour)}")
            print(f"Hourly assets : {hourly_assets[hour]}")
            print(f"length of hourly assets : {len(hourly_assets[hour])}")"""
            # check if there are duplicates
            if len(hourly_assets[hour]) != len(set(hourly_assets[hour])) and hourly_assets[hour] != []:
                print(f"Duplicate assets in hour {hour}")
                # print the duplicates
                print(f"length of hourly assets : {len(hourly_assets[hour])}")
                print(f"length of set of hourly assets : {len(set(hourly_assets[hour]))}")
                return False
    return hourly_assets
    

In [71]:
ha = feasibility_check(bids, L)

In [73]:
[len(ha[h]) for h in set(hours)]

[897,
 897,
 495,
 897,
 889,
 495,
 0,
 1661,
 897,
 1633,
 495,
 1112,
 889,
 889,
 889,
 897,
 897,
 897,
 1101,
 495,
 897,
 897,
 0,
 897,
 1661,
 1633,
 889,
 495,
 495,
 0,
 889,
 889,
 897,
 495,
 897,
 495,
 1101,
 897,
 0,
 897,
 889,
 889,
 495,
 495,
 889,
 1112,
 1112,
 889]